https://python.quantecon.org/qr_decomp.html

In [1]:
import numpy as np
from scipy.linalg import qr

In [2]:
def QR_Decomposition(A):
    n, m = A.shape # get the shape of A

    Q = np.empty((n, n)) # initialize matrix Q
    u = np.empty((n, n)) # initialize matrix u

    u[:, 0] = A[:, 0]
    Q[:, 0] = u[:, 0] / np.linalg.norm(u[:, 0])

    for i in range(1, n):

        u[:, i] = A[:, i]
        for j in range(i):
            u[:, i] -= (A[:, i] @ Q[:, j]) * Q[:, j] # get each u vector

        Q[:, i] = u[:, i] / np.linalg.norm(u[:, i]) # compute each e vetor

    R = np.zeros((n, m))
    for i in range(n):
        for j in range(i, m):
            R[i, j] = A[:, j] @ Q[:, i]

    return Q, R

In [3]:
def diag_sign(A):
    "Compute the signs of the diagonal of matrix A"

    D = np.diag(np.sign(np.diag(A)))

    return D

def adjust_sign(Q, R):
    """
    Adjust the signs of the columns in Q and rows in R to
    impose positive diagonal of Q
    """

    D = diag_sign(Q)

    Q[:, :] = Q @ D
    R[:, :] = D @ R

    return Q, R

In [4]:
A = np.array([[1.0, 1.0, 0.0], [1.0, 0.0, 1.0], [0.0, 1.0, 1.0]])
# A = np.array([[1.0, 0.5, 0.2], [0.5, 0.5, 1.0], [0.0, 1.0, 1.0]])
# A = np.array([[1.0, 0.5, 0.2], [0.5, 0.5, 1.0]])

A

array([[1., 1., 0.],
       [1., 0., 1.],
       [0., 1., 1.]])

In [5]:
Q, R = adjust_sign(*QR_Decomposition(A))

In [6]:
Q

array([[ 0.70710678, -0.40824829, -0.57735027],
       [ 0.70710678,  0.40824829,  0.57735027],
       [ 0.        , -0.81649658,  0.57735027]])

In [7]:
R

array([[ 1.41421356,  0.70710678,  0.70710678],
       [ 0.        , -1.22474487, -0.40824829],
       [ 0.        ,  0.        ,  1.15470054]])

In [8]:
Q_scipy, R_scipy = adjust_sign(*qr(A))

In [9]:
print('Our Q: \n', Q)
print('\n')
print('Scipy Q: \n', Q_scipy)

Our Q: 
 [[ 0.70710678 -0.40824829 -0.57735027]
 [ 0.70710678  0.40824829  0.57735027]
 [ 0.         -0.81649658  0.57735027]]


Scipy Q: 
 [[ 0.70710678 -0.40824829 -0.57735027]
 [ 0.70710678  0.40824829  0.57735027]
 [ 0.         -0.81649658  0.57735027]]


In [10]:
print('Our R: \n', R)
print('\n')
print('Scipy R: \n', R_scipy)

Our R: 
 [[ 1.41421356  0.70710678  0.70710678]
 [ 0.         -1.22474487 -0.40824829]
 [ 0.          0.          1.15470054]]


Scipy R: 
 [[ 1.41421356  0.70710678  0.70710678]
 [ 0.         -1.22474487 -0.40824829]
 [ 0.          0.          1.15470054]]


In [11]:
A = np.array([[1, 3, 4], [2, 0, 9]])

In [12]:
Q, R = adjust_sign(*QR_Decomposition(A))
Q, R

(array([[ 0.4472136 , -0.89442719],
        [ 0.89442719,  0.4472136 ]]),
 array([[ 2.23606798,  1.34164079,  9.8386991 ],
        [ 0.        , -2.68328157,  0.4472136 ]]))

In [13]:
Q_scipy, R_scipy = adjust_sign(*qr(A))
Q_scipy, R_scipy

(array([[ 0.4472136 , -0.89442719],
        [ 0.89442719,  0.4472136 ]]),
 array([[ 2.23606798,  1.34164079,  9.8386991 ],
        [ 0.        , -2.68328157,  0.4472136 ]]))

In [14]:
def QR_eigvals(A, tol=1e-12, maxiter=1000):
    "Find the eigenvalues of A using QR decomposition."

    A_old = np.copy(A)
    A_new = np.copy(A)

    diff = np.inf
    i = 0
    while (diff > tol) and (i < maxiter):
        A_old[:, :] = A_new
        Q, R = QR_Decomposition(A_old)

        A_new[:, :] = R @ Q

        diff = np.abs(A_new - A_old).max()
        i += 1

    eigvals = np.diag(A_new)

    return eigvals

In [15]:
# experiment this with one random A matrix
A = np.random.random((3, 3))

In [16]:
sorted(QR_eigvals(A))

[-0.5578464131831805, 0.2817097148158302, 1.3228104693388458]

In [17]:
sorted(np.linalg.eigvals(A))

[-0.5578464131831808, 0.281709714815838, 1.3228104693388456]

In [18]:
k = 5
n = 1000

# generate some random moments
𝜇 = np.random.random(size=k)
C = np.random.random((k, k))
Σ = C.T @ C

In [19]:
# X is random matrix where each column follows multivariate normal dist.
X = np.random.multivariate_normal(𝜇, Σ, size=n)

In [20]:
X.shape

(1000, 5)

In [21]:
Q, R = adjust_sign(*QR_Decomposition(X.T))

In [22]:
Q.shape, R.shape

((5, 5), (5, 1000))

In [23]:
RR = R @ R.T

𝜆, P_tilde = np.linalg.eigh(RR)
Λ = np.diag(𝜆)

In [24]:
XX = X.T @ X

𝜆_hat, P = np.linalg.eigh(XX)
Λ_hat = np.diag(𝜆_hat)

In [25]:
𝜆, 𝜆_hat

(array([  31.3818442 ,  259.62240203,  415.69522764,  814.32987453,
        8304.60879681]),
 array([  31.3818442 ,  259.62240203,  415.69522764,  814.32987453,
        8304.60879681]))

In [26]:
QP_tilde = Q @ P_tilde

np.abs(P @ diag_sign(P) - QP_tilde @ diag_sign(QP_tilde)).max()

1.9984014443252818e-15

In [27]:
QPΛPQ = Q @ P_tilde @ Λ @ P_tilde.T @ Q.T

In [28]:
np.abs(QPΛPQ - XX).max()

7.275957614183426e-12